<a href="https://colab.research.google.com/github/StrayBeing/AI_Lab/blob/main/Lab3_Rozpoznawanie_Zdjec_dataset_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [58]:
mnist_dataset,minst_info = tfds.load(name='mnist',with_info=True,as_supervised=True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']
num_validation_samples = 0.1 * minst_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples,tf.int64)
num_test_samples = minst_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)

In [59]:
def scale(image,label):
  image=tf.cast(image,tf.float32)
  image/=255.
  return image,label
  scaled_train_and_validation_data=mnist_train.map(scale)
  test_data=mnist_test.map(scale)
scaled_train_and_validation_data=mnist_train.map(scale)
test_data=mnist_test.map(scale)

In [60]:
BUFFER_SIZE = 1000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [61]:
BATCH_SIZE =100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)
validation_inputs, validation_targets = next(iter(validation_data))
print(validation_inputs.shape,validation_targets.shape)

(6000, 28, 28, 1) (6000,)


In [62]:
input_size = 784
output_size =10
hidden_layer_size = 50
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(output_size,activation='softmax')
])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [63]:
NUM_EPOCHS= 30
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
model.fit(train_data,epochs=NUM_EPOCHS,callbacks=[early_stopping],validation_data=(validation_inputs,validation_targets),verbose=1)

Epoch 1/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.7861 - loss: 0.7380 - val_accuracy: 0.9338 - val_loss: 0.2320
Epoch 2/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9437 - loss: 0.1962 - val_accuracy: 0.9468 - val_loss: 0.1743
Epoch 3/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9566 - loss: 0.1468 - val_accuracy: 0.9548 - val_loss: 0.1463
Epoch 4/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9650 - loss: 0.1183 - val_accuracy: 0.9587 - val_loss: 0.1349
Epoch 5/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9717 - loss: 0.0973 - val_accuracy: 0.9623 - val_loss: 0.1247
Epoch 6/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9759 - loss: 0.0820 - val_accuracy: 0.9638 - val_loss: 0.1223
Epoch 7/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9794 - loss: 0.0710 - val_accuracy: 0.9632 - val_loss: 0.1277
Epoch 8/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9814 - loss: 0.0649 - val_accuracy: 0

In [64]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 663ms/step - accuracy: 0.9715 - loss: 0.1024
Test loss: 0.10. Test accuracy: 97.15%


In [96]:
import cv2
import numpy as np
import tensorflow as tf
liczba_obrazkow = 9
for i in range(1, liczba_obrazkow + 1):
      filename = f'/content/sample_data/{i}.png'
      try:
          img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
          if img is None:
            print(f"Nie można załadować pliku: {filename}")
            continue
          img_resized = cv2.resize(img, (28, 28))
          img_normalized = img_resized / 255.0
          img_reshaped = img_normalized.reshape(1, 28, 28, 1)
          prediction = model.predict(img_reshaped, verbose=0)
          predicted_class = np.argmax(prediction)
          confidence = np.max(prediction) * 100
          print(f"{filename}: Model przewiduje {predicted_class} z pewnością {confidence:.2f}%")
      except Exception as e:
          print(f"Coś poszło nie tak z plikiem {filename}: {e}")

/content/sample_data/1.png: Model przewiduje 3 z pewnością 99.99%
/content/sample_data/2.png: Model przewiduje 8 z pewnością 91.83%
/content/sample_data/3.png: Model przewiduje 5 z pewnością 96.46%
/content/sample_data/4.png: Model przewiduje 6 z pewnością 99.38%
/content/sample_data/5.png: Model przewiduje 2 z pewnością 99.86%
/content/sample_data/6.png: Model przewiduje 0 z pewnością 90.34%
/content/sample_data/7.png: Model przewiduje 8 z pewnością 99.82%
/content/sample_data/8.png: Model przewiduje 3 z pewnością 89.05%
/content/sample_data/9.png: Model przewiduje 6 z pewnością 97.41%
